In [19]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/ubuntu/varios/skforecast


In [2]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("/home/ubuntu/varios/skforecast/skforecast/model_selection/tests")
series_item_sales = pd.read_parquet(THIS_DIR/'fixture_multi_series_items_sales.parquet')
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [ ]:
def test_output_bayesian_search_forecaster_multiseries_series_and_exog_dict_with_mocked_with_window_features():
    """
    Test output of bayesian_search_forecaster_multiseries in ForecasterRecursiveMultiSeries
    when series and exog are dictionaries (mocked done in Skforecast v0.12.0).
    """

    window_features = RollingFeatures(
        stats=['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
        window_sizes=3,
    )

    forecaster = ForecasterRecursiveMultiSeries(
        regressor=LGBMRegressor(
            n_estimators=2, random_state=123, verbose=-1, max_depth=2
        ),
        lags=14,
        window_features=window_features,
        encoding="ordinal",
        dropna_from_series=False,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
    )
    cv = TimeSeriesFold(
            initial_train_size = len(series_dict_train["id_1000"]),
            steps              = 24,
            refit              = False,
            fixed_train_size   = True
    )
    lags_grid = [[5], [1, 7, 14]]

    def search_space(trial):
        search_space = {
            "n_estimators": trial.suggest_int("n_estimators", 2, 5),
            "max_depth": trial.suggest_int("max_depth", 2, 5),
            "lags": trial.suggest_categorical("lags", lags_grid),
        }

        return search_space

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

        results_search, _ = bayesian_search_forecaster_multiseries(
            forecaster         = forecaster,
            series             = series_dict,
            exog               = exog_dict,
            cv                 = cv,
            search_space       = search_space,
            metric             = "mean_absolute_error",
            aggregate_metric   = "weighted_average",
            n_trials           = 3,
            return_best        = False,
            show_progress      = False,
            verbose            = False,
            suppress_warnings  = True,
        )

    expected = pd.DataFrame(
        np.array(
            [
                [
                    list(["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"]),
                    np.array([1, 7, 14]),
                    {"n_estimators": 4, "max_depth": 3},
                    709.8836514262415,
                    4,
                    3,
                ],
                [
                    list(["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"]),
                    np.array([1, 7, 14]),
                    {"n_estimators": 3, "max_depth": 3},
                    721.1848222120482,
                    3,
                    3,
                ],
                [
                    list(["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"]),
                    np.array([5]),
                    {"n_estimators": 4, "max_depth": 3},
                    754.3537196425694,
                    4,
                    3,
                ],
            ],
            dtype=object,
        ),
        columns=[
            "levels",
            "lags",
            "params",
            "mean_absolute_error__weighted_average",
            "n_estimators",
            "max_depth",
        ],
        index=pd.Index([0, 1, 2], dtype="int64"),
    ).astype(
        {
            "mean_absolute_error__weighted_average": float,
            "n_estimators": int,
            "max_depth": int,
        }
    )

    results_search = results_search.astype(
        {
            "mean_absolute_error__weighted_average": float,
            "n_estimators": int,
            "max_depth": int,
        }
    )

    pd.testing.assert_frame_equal(expected, results_search)

In [9]:
window_features = RollingFeatures(
    stats=['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes=3,
)

forecaster = ForecasterRecursiveMultiSeries(
    regressor=LGBMRegressor(
        n_estimators=2, random_state=123, verbose=-1, max_depth=2
    ),
    lags=14,
    window_features=window_features,
    encoding="ordinal",
    dropna_from_series=False,
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)
cv = TimeSeriesFold(
        initial_train_size = len(series_dict_train["id_1000"]),
        steps              = 24,
        refit              = False,
        fixed_train_size   = True
)
lags_grid = [[5], [1, 7, 14]]

def search_space(trial):
    search_space = {
        "n_estimators": trial.suggest_int("n_estimators", 2, 5),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "lags": trial.suggest_categorical("lags", lags_grid),
    }

    return search_space

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

    results_search, _ = bayesian_search_forecaster_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        cv                 = cv,
        search_space       = search_space,
        metric             = "mean_absolute_error",
        aggregate_metric   = "weighted_average",
        n_trials           = 3,
        return_best        = False,
        show_progress      = False,
        verbose            = False,
        suppress_warnings  = True,
    )

  0%|          | 0/3 [00:00<?, ?it/s]

[W 2024-10-27 15:47:20,296] Trial 0 failed with parameters: {'n_estimators': 4, 'max_depth': 3, 'lags': [1, 7, 14]} because of the following error: ZeroDivisionError('division by zero').
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/skforecast_14_py12/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/home/ubuntu/anaconda3/envs/skforecast_14_py12/lib/python3.12/site-packages/skforecast/model_selection/_search.py", line 1879, in _objective
    metrics, _ = backtesting_forecaster_multiseries(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/anaconda3/envs/skforecast_14_py12/lib/python3.12/site-packages/skforecast/model_selection/_validation.py", line 905, in backtesting_forecaster_multiseries
    metrics_levels, backtest_predictions = _backtesting_forecaster_multiseries(
                                           ^^^^^^^^^^^^^^^^^^^^

ZeroDivisionError: division by zero

In [8]:
def test_results_output_bayesian_search_forecaster_multiseries_ForecasterAutoregMultiSeries_with_window_features():
    """
    Test output of bayesian_search_forecaster_multiseries in 
    ForecasterRecursiveMultiSeries with mocked (mocked done in Skforecast v0.12.0).
    """
    window_features = RollingFeatures(
        stats=['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
        window_sizes=3,
    )

    forecaster = ForecasterRecursiveMultiSeries(
                     regressor = Ridge(random_state=123),
                     lags      = 2,
                     window_features = window_features,
                     encoding  = 'onehot',
                     transformer_series = StandardScaler()
                 )
    cv = TimeSeriesFold(
            initial_train_size = len(series[:-12]),
            steps              = 3,
            refit              = False,
            fixed_train_size   = True
    )

    def search_space(trial):
        search_space  = {
            'alpha': trial.suggest_float('alpha', 1e-2, 1.0),
            'lags': trial.suggest_categorical('lags', [2, 4])
        }

        return search_space

    results = bayesian_search_forecaster_multiseries(
                  forecaster         = forecaster,
                  series             = series,
                  cv                 = cv,
                  search_space       = search_space,
                  metric             = 'mean_absolute_error',
                  aggregate_metric   = 'weighted_average',
                  n_trials           = 10,
                  random_state       = 123,
                  return_best        = False,
                  verbose            = False,
              )[0]
    
    expected_results = pd.DataFrame({
        'levels': [
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
            ['l1', 'l2'],
        ],
        'lags': [
            np.array([1, 2]),
            np.array([1, 2, 3, 4]),
            np.array([1, 2]),
            np.array([1, 2]),
            np.array([1, 2]),
            np.array([1, 2, 3, 4]),
            np.array([1, 2]),
            np.array([1, 2, 3, 4]),
            np.array([1, 2, 3, 4]),
            np.array([1, 2, 3, 4]),
        ],
        'params': [
            {'alpha': 0.5558016213920624},
            {'alpha': 0.23598059857016607},
            {'alpha': 0.6995044937418831},
            {'alpha': 0.7406154516747153},
            {'alpha': 0.8509374761370117},
            {'alpha': 0.7252189487445193},
            {'alpha': 0.9809565564007693},
            {'alpha': 0.53623586010342},
            {'alpha': 0.4441865222328282},
            {'alpha': 0.398196343012209},
        ],
        'mean_absolute_error__weighted_average': [
            0.261692844251679,
            0.2643237951473078,
            0.26861473009966413,
            0.27087715575521204,
            0.2777007069915752,
            0.27967030912996776,
            0.2874984215783917,
            0.2988729113427532,
            0.345749311164626,
            0.5104999292169147,
        ],
        'alpha': [
            0.5558016213920624,
            0.23598059857016607,
            0.6995044937418831,
            0.7406154516747153,
            0.8509374761370117,
            0.7252189487445193,
            0.9809565564007693,
            0.53623586010342,
            0.4441865222328282,
            0.398196343012209,
        ],
    })

    pd.testing.assert_frame_equal(results, expected_results)


test_results_output_bayesian_search_forecaster_multiseries_ForecasterAutoregMultiSeries_with_window_features()

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
window_features = RollingFeatures(
    stats=['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes=3,
)

forecaster = ForecasterRecursiveMultiSeries(
                    regressor = Ridge(random_state=123),
                    lags      = 2,
                    window_features = window_features,
                    encoding  = 'onehot',
                    transformer_series = StandardScaler()
                )
cv = TimeSeriesFold(
        initial_train_size = len(series[:-12]),
        steps              = 3,
        refit              = False,
        fixed_train_size   = True
)

def search_space(trial):
    search_space  = {
        'alpha': trial.suggest_float('alpha', 1e-2, 1.0),
        'lags': trial.suggest_categorical('lags', [2, 4])
    }

    return search_space

results = bayesian_search_forecaster_multiseries(
                forecaster         = forecaster,
                series             = series,
                cv                 = cv,
                search_space       = search_space,
                metric             = 'mean_absolute_error',
                aggregate_metric   = 'weighted_average',
                n_trials           = 10,
                random_state       = 123,
                return_best        = False,
                verbose            = False,
            )[0]

results.to_dict(orient='list')

  0%|          | 0/10 [00:00<?, ?it/s]

{'levels': [['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2']],
 'lags': [array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4])],
 'params': [{'alpha': 0.5558016213920624},
  {'alpha': 0.23598059857016607},
  {'alpha': 0.6995044937418831},
  {'alpha': 0.7406154516747153},
  {'alpha': 0.8509374761370117},
  {'alpha': 0.7252189487445193},
  {'alpha': 0.9809565564007693},
  {'alpha': 0.53623586010342},
  {'alpha': 0.4441865222328282},
  {'alpha': 0.398196343012209}],
 'mean_absolute_error__weighted_average': [0.261692844251679,
  0.2643237951473078,
  0.26861473009966413,
  0.27087715575521204,
  0.2777007069915752,
  0.27967030912996776,
  0.2874984215783917,
  0.2988729113427532,
  0.345749311164626,
  0.5104999292169147],
 'alpha': [0.55580

In [7]:
expected_results = pd.DataFrame({
    'levels': [
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
        ['l1', 'l2'],
    ],
    'lags': [
        np.array([1, 2]),
        np.array([1, 2, 3, 4]),
        np.array([1, 2]),
        np.array([1, 2]),
        np.array([1, 2]),
        np.array([1, 2, 3, 4]),
        np.array([1, 2]),
        np.array([1, 2, 3, 4]),
        np.array([1, 2, 3, 4]),
        np.array([1, 2, 3, 4]),
    ],
    'params': [
        {'alpha': 0.5558016213920624},
        {'alpha': 0.23598059857016607},
        {'alpha': 0.6995044937418831},
        {'alpha': 0.7406154516747153},
        {'alpha': 0.8509374761370117},
        {'alpha': 0.7252189487445193},
        {'alpha': 0.9809565564007693},
        {'alpha': 0.53623586010342},
        {'alpha': 0.4441865222328282},
        {'alpha': 0.398196343012209},
    ],
    'mean_absolute_error__weighted_average': [
        0.261692844251679,
        0.2643237951473078,
        0.26861473009966413,
        0.27087715575521204,
        0.2777007069915752,
        0.27967030912996776,
        0.2874984215783917,
        0.2988729113427532,
        0.345749311164626,
        0.5104999292169147,
    ],
    'alpha': [
        0.5558016213920624,
        0.23598059857016607,
        0.6995044937418831,
        0.7406154516747153,
        0.8509374761370117,
        0.7252189487445193,
        0.9809565564007693,
        0.53623586010342,
        0.4441865222328282,
        0.398196343012209,
    ],
})


In [10]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
# from skopt.space import Real
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection._search import bayesian_search_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y

In [15]:
def test_results_output_bayesian_search_forecaster_optuna_ForecasterAutoreg_window_features_with_mocked():
    """
    Test output of bayesian_search_forecaster in ForecasterRecursive with window features 
    using mocked using optuna (mocked done in Skforecast v0.4.3).
    """
    window_features = RollingFeatures(
        stats        = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
        window_sizes = 3,
    )
    forecaster = ForecasterRecursive(
                     regressor      = Ridge(random_state=123),
                     lags           = 4,
                    window_features = window_features,
                 )

    cv = TimeSeriesFold(
            steps                 = 3,
            initial_train_size    = len(y[:-12]),
            window_size           = None,
            differentiation       = None,
            refit                 = True,
            fixed_train_size      = True,
            gap                   = 0,
            skip_folds            = None,
            allow_incomplete_fold = True,
            return_all_indexes    = False,
        )
    
    def search_space(trial):  # pragma: no cover
        search_space  = {'alpha': trial.suggest_float('alpha', 1e-2, 1.0)}
        return search_space

    results = bayesian_search_forecaster(
                  forecaster         = forecaster,
                  y                  = y,
                  cv                 = cv,
                  search_space       = search_space,
                  metric             = 'mean_absolute_error',
                  n_trials           = 10,
                  random_state       = 123,
                  return_best        = False,
                  verbose            = False,
              )[0]
    
    expected_results = pd.DataFrame({
            'lags': [
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2, 3, 4]),
            ],
            'params': [
                {'alpha': 0.9809565564007693},
                {'alpha': 0.7222742800877074},
                {'alpha': 0.6995044937418831},
                {'alpha': 0.6879814411990146},
                {'alpha': 0.5558016213920624},
                {'alpha': 0.48612258246951734},
                {'alpha': 0.42887539552321635},
                {'alpha': 0.398196343012209},
                {'alpha': 0.29327794160087567},
                {'alpha': 0.2345829390285611},
            ],
            'mean_absolute_error': [
                0.23783372219201282,
                0.24038797813509474,
                0.24065731430061374,
                0.24079724452316387,
                0.24261647067658373,
                0.24378706429210686,
                0.24490876891227564,
                0.24558757754181923,
                0.24851969600385504,
                0.2508328027649236,
            ],
            'alpha': [
                0.9809565564007693,
                0.7222742800877074,
                0.6995044937418831,
                0.6879814411990146,
                0.5558016213920624,
                0.48612258246951734,
                0.42887539552321635,
                0.398196343012209,
                0.29327794160087567,
                0.2345829390285611,
            ],
    })

    pd.testing.assert_frame_equal(results, expected_results)

test_results_output_bayesian_search_forecaster_optuna_ForecasterAutoreg_window_features_with_mocked()

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
window_features = RollingFeatures(
    stats        = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes = 3,
)
forecaster = ForecasterRecursive(
                    regressor      = Ridge(random_state=123),
                    lags           = 4,
                window_features = window_features,
                )

cv = TimeSeriesFold(
        steps                 = 3,
        initial_train_size    = len(y[:-12]),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = True,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )

def search_space(trial):  # pragma: no cover
    search_space  = {'alpha': trial.suggest_float('alpha', 1e-2, 1.0)}
    return search_space

results = bayesian_search_forecaster(
                forecaster         = forecaster,
                y                  = y,
                cv                 = cv,
                search_space       = search_space,
                metric             = 'mean_absolute_error',
                n_trials           = 10,
                random_state       = 123,
                return_best        = False,
                verbose            = False,
            )[0]

results.to_dict(orient='list')

  0%|          | 0/10 [00:00<?, ?it/s]

{'lags': [array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4]),
  array([1, 2, 3, 4])],
 'params': [{'alpha': 0.9809565564007693},
  {'alpha': 0.7222742800877074},
  {'alpha': 0.6995044937418831},
  {'alpha': 0.6879814411990146},
  {'alpha': 0.5558016213920624},
  {'alpha': 0.48612258246951734},
  {'alpha': 0.42887539552321635},
  {'alpha': 0.398196343012209},
  {'alpha': 0.29327794160087567},
  {'alpha': 0.2345829390285611}],
 'mean_absolute_error': [0.23783372219201282,
  0.24038797813509474,
  0.24065731430061374,
  0.24079724452316387,
  0.24261647067658373,
  0.24378706429210686,
  0.24490876891227564,
  0.24558757754181923,
  0.24851969600385504,
  0.2508328027649236],
 'alpha': [0.9809565564007693,
  0.7222742800877074,
  0.6995044937418831,
  0.6879814411990146,
  0.5558016213920624,
  0.48612258246951734,
  0.42887539552321635,
